# Разные задачки

Попрактикуемся по разным темам

## Сумма цифр числа рекурсией

Дано число произвольного размера. Посчитайте рекурсивно сумму его цифр

In [ ]:
def sum_of_digit(n, val):
 
    # base case
    
    # your code
 
    return 

## Run-Length Encoding

RLE -- это один из алгоритмом сжатия последовательностей.

Пусть у вас есть строка `aaaaaabbbcbbddde`, ее можно представить в виде `[(a, 6), (b, 3), (c, 1), (b, 2), (d, 3), (e, 1)]` 

In [13]:
s = 'aaaaaabbbcbbddde'

**Напишем обычную версию**

In [12]:
def do_rle(s: str) -> list[tuple]:
    encoded_s = []
    # your code
    
    
    return encoded_s

In [14]:
do_rle(s)

[('a', 6), ('b', 3), ('c', 1), ('b', 2), ('d', 3), ('e', 1)]

**Рекурсивная версия**

In [7]:
def do_rle_recursive(s: str, encoded_s: list[tuple]) -> list[tuple]:
    
    # your base case
    
    # your code
    
    return 

In [8]:
s = 'aaaaaabbbcbbddde'

do_rle_recursive(s, [])

[('a', 6)] 6
[('a', 6), ('b', 3)] 3
[('a', 6), ('b', 3), ('c', 1)] 1
[('a', 6), ('b', 3), ('c', 1), ('b', 2)] 2
[('a', 6), ('b', 3), ('c', 1), ('b', 2), ('d', 3)] 3
[('a', 6), ('b', 3), ('c', 1), ('b', 2), ('d', 3), ('e', 1)] 1


[('a', 6), ('b', 3), ('c', 1), ('b', 2), ('d', 3), ('e', 1)]

## Уникальные слова в тексте

Вывести самые популярные слова в тексте и их количество

Если дан не текст, а список из слов через запятую, то как можно решить задачу? А как превратить текст в список из слов?

In [11]:
sample_text = "The secret menu reveals a business model that goes beyond a right to repair issue, Sullivan argues. It represents, as he describes it, nothing short of a milkshake shakedown: Sell franchisees a complicated and fragile machine. Prevent them from figuring out why it constantly breaks. Take a cut of the distributors’ profit from the repairs. “It’s a huge money maker to have a customer that’s purposefully, intentionally blind and unable to make very fundamental changes to their own equipment,” Sullivan says. And McDonald’s presides over all of it, he says, insisting on loyalty to its longtime supplier. (Resist the McDonald’s monarchy on decisions like equipment, and the corporation can end a restaurant’s lease on the literal ground beneath it, which McDonald's owns under its franchise agreement.)"

In [34]:
import re
from collections import Counter

In [ ]:
Counter(re.split('[,.():\s]+', sample_text.lower().strip()))

## Общие элементы двух списков (однострочник)

Даны списки a и b. Вывести общие элементы в них

In [51]:
a = [1, 2, 3, 4, 7, 8, 9, 7, 7]
b = [5, 4, 6, 7, 10, 11, 12, 7, 7]

# your code

{4, 7}

In [1]:
from collections import Counter

# your code

## Номер появления слова

Дан список из слов. Получить список с их номерами по порядку их появления.

```
word_list = ['two', 'one', 'three', 'three', 'two', 'three']
fun(word_list) -> [1, 1, 1, 2, 2, 3]
```

In [53]:
word_list = ['two', 'one', 'three', 'three', 'two', 'three']

In [56]:
from collections import defaultdict

# your code

[1, 1, 1, 2, 2, 3]

## Непрерывный массив суммой N

Дано: некоторый массив целых чисел и число N

Задача: вывести такой непрерывный подмассив, сумма элементов которого равна N. Если таких несколько, то вывести максимальную длину. Если не существует, то вывести 0

```
Пример:

1.
Дано: [1, 2, 3, 4, -1, 2, -1, 4, 0], N = 4
Подмассивы с суммой 4: [4], [4, -1, 2, -1], [-1, 2, -1, 4, 0]
Ответ: 5

2.
Дано: [1, 2, 3], N = 10
Ответ: 0
```

In [94]:
from itertools import accumulate

def find_n_subarray(arr: list, n: int) -> int:
    
    # your code
            
    return max_subarr_len

In [98]:
arr = [1, 2, 3, 4, -1, 2, -1, 4, 0]

find_n_subarray(arr, 4)

5

# Продолжаем про декораторы 

## Замена исходной функции

Функция, возвращаемая декоратором, заменяет декорируемую

In [136]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [139]:
def target():
    print('running target()')

In [140]:
@deco
def target():
    print('running target()')

In [141]:
target()

running inner()


In [143]:
target  # now reference to inner 

<function __main__.deco.<locals>.inner()>

## Running at import time

Key feature декораторов в том, что они применяются сразу после объявления декорируемой функции во время загрузки модуля в Python 

In [147]:
registry = []

def register(func):
    print(f'running register {func}')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')
    
def run():
    print('running run()')
    print('registry', registry)
    f1()
    f2()
    f3()

running register <function f1 at 0x7f6c72626f80>
running register <function f2 at 0x7f6c72626ef0>


In [148]:
run()

running run()
registry [<function f1 at 0x7f6c72626f80>, <function f2 at 0x7f6c72626ef0>]
running f1()
running f2()
running f3()


## Декоратор-логгер

### logging

 Модуль `logging` содержит набор функций для логирования различного поведения в вашей программе. Например, для понимания, что ваша программа работает как ожидается, или там идет что-то не так

Есть разные уровни логирования: debug, info, warning, error, critical

In [1]:
import logging

In [2]:
logging.info('Working as expected')
logging.warning('Smth might go wrong, check this!')

Можно логировать в файл (надо начать новую сессию в Python)

In [15]:
import os
import logging

log_filename = os.path.join('data', 'example.log')
log_level = logging.DEBUG

logging.basicConfig(
    filename=log_filename,
    encoding='utf-8',
    level=log_level,
    format='%(asctime)s %(message)s'  # add time
)

logging.debug('One very thorough message to save in our log')
logging.info('Just some info, also in log')
logging.warning('Something might be wrong')
logging.error('Logging our failure')

In [2]:
! cat data/example.log

2022-09-29 03:04:13,025 One very thorough message to save in our log
2022-09-29 03:04:13,025 Just some info, also in log
2022-09-29 03:04:13,025 Something might be wrong
2022-09-29 03:04:13,025 Logging our failure


### Напишем log-декоратор

In [103]:
import logging
import sys

from functools import wraps, partial

In [155]:
logger = logging.getLogger(name='log')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(stream=sys.stdout))

In [159]:
def logged(func=None,
           *,
           log_level=logging.INFO,
           log_name=None,
           log_message=None):
    
    # code
    
    return wrapper

In [160]:
@logged(log_name='log')
def add(x, y):
    return x + y

In [161]:
add(1, 2)

add
add
add
add
add
add
add
add
add
add
add
add
add
add
add
add


INFO:log:add


3

Протестируем с декоратором с прошлых семинаров

In [164]:
def clock(func):
    @wraps(func)
    def clocked(*args):
        t_start = time.time()
        res = func(*args)
        total_time = time.time() - t_start
        print(f'{func.__name__}({args}) -> {res} executed in {total_time:.2f}s')
        return res
    return clocked

In [168]:
@logged(log_name='log')
@clock
def add(x, y):
    time.sleep(2)
    return x + y

In [169]:
add(10, 10**10)

add
add
add
add
add
add
add
add
add
add
add
add
add
add
add
add


INFO:log:add


add((10, 10000000000)) -> 10000000010 executed in 2.00s


10000000010

## Декоратор -- assert типов (3 балла)

Мы уже говорили про type hints, а также про то, что никаких ошибок без использования сторонних модулей некорректные типы нам не дадут. Давайте реализуем логику проверки типов с помощью декоратора.

Типы могут быть заданы как стандартными названиями, так и с использованыем модуля typing. Для Any пропускайте проверку

In [11]:
from typing import Any

In [9]:
@check_types
def func(a: int, b: str) -> str:
    return a * b

Стоит выбросить ValueError, если есть несовпадение между реальным и ожидаемым типом у какой-то из переданных переменных или у возвращаемого значения